# Sleep Analysis

A study of sleep efficiency, and sleep patterns.

## About Dataset

The dataset contains information about a group of test subjects and their sleep patterns. Each test subject is identified by a unique "Subject ID" and their age and gender are also recorded. The "Bedtime" and "Wakeup time" features indicate when each subject goes to bed and wakes up each day, and the "Sleep duration" feature records the total amount of time each subject slept in hours. The "Sleep efficiency" feature is a measure of the proportion of time spent in bed that is actually spent asleep. The "REM sleep percentage", "Deep sleep percentage", and "Light sleep percentage" features indicate the amount of time each subject spent in each stage of sleep. The "Awakenings" feature records the number of times each subject wakes up during the night. Additionally, the dataset includes information about each subject's caffeine and alcohol consumption in the 24 hours prior to bedtime, their smoking status, and their exercise frequency.

There are 452 observations of 14 variables.  Awakenings has 20 missing values, Caffeine consumption has 25 missing values, Alcohol consumption has 14 missing values, Exercise frequency has 6 missing values.

## Objective

A typical adult needs 1.5 to 2 hours of deep sleep and 1.5 to 2 hours of REM sleep. There is no minimum amout of light sleep defined for a quality night of sleep. A label will be created called Sleep Quality that defines the provides percentage of required sleep achieved. In the case of someone with at least 1.5 hours of deep sleep and 1.5 hours of REM sleep they will receive a Sleep Quality rating of 100%. If someone has 1 hour of deep sleep and 1.5 hours of REM sleep they will receive a Sleep Quality rating of 83%.

$Sleep Quality = \frac{\min\{\frac{Deep Sleep}{1.5}, 1\} + \min\{\frac{REM Sleep}{1.5}, 1\}}{2}*100$

The objective of this analysis will be to determine which features impact the Sleep Quality the most and to be able to predict the sleep quality.

In [1]:
from IPython.display import clear_output

In [2]:
%%time
%pip install kagglehub
%pip install ipywidgets
%pip install pandas
%pip install seaborn
%pip install plotly
%pip install nbformat>=4.2.0
clear_output()

CPU times: total: 78.1 ms
Wall time: 6.06 s


In [3]:
import kagglehub
import shutil
import os

# Download latest version
path = kagglehub.dataset_download("equilibriumm/sleep-efficiency")
destination = r"data"
for item in os.listdir(path):
    src = os.path.join(path, item)
    dest = os.path.join(destination, item)
    if os.path.isdir(src):
        shutil.copytree(src, dest, dirs_exist_ok=True)
    else:
        shutil.copy2(src, dest)

In [14]:
import pandas as pd

sleep_df = pd.read_csv(r"data\Sleep_Efficiency.csv").set_index("ID")
sleep_df["Bedtime"] = pd.to_datetime(sleep_df["Bedtime"]).dt.time
sleep_df["Wakeup time"] = pd.to_datetime(sleep_df["Wakeup time"]).dt.time
sleep_df

,Age,Gender,Bedtime,Wakeup time,Sleep duration,Sleep efficiency,REM sleep percentage,Deep sleep percentage,Light sleep percentage,Awakenings,Caffeine consumption,Alcohol consumption,Smoking status,Exercise frequency
ID,,,,,,,,,,,,,,
1,65,Female,01:00:00,07:00:00,6.0,0.88,18,70,12,0.0,0.0,0.0,Yes,3.0
2,69,Male,02:00:00,09:00:00,7.0,0.66,19,28,53,3.0,0.0,3.0,Yes,3.0
3,40,Female,21:30:00,05:30:00,8.0,0.89,20,70,10,1.0,0.0,0.0,No,3.0
4,40,Female,02:30:00,08:30:00,6.0,0.51,23,25,52,3.0,50.0,5.0,Yes,1.0
5,57,Male,01:00:00,09:00:00,8.0,0.76,27,55,18,3.0,0.0,3.0,No,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
448,27,Female,22:00:00,05:30:00,7.5,0.91,22,57,21,0.0,0.0,0.0,No,5.0
449,52,Male,21:00:00,03:00:00,6.0,0.74,28,57,15,4.0,25.0,0.0,No,3.0
450,40,Female,23:00:00,07:30:00,8.5,0.55,20,32,48,1.0,NaN,3.0,Yes,0.0


In [15]:
sleep_df.describe()

,Age,Sleep duration,Sleep efficiency,REM sleep percentage,Deep sleep percentage,Light sleep percentage,Awakenings,Caffeine consumption,Alcohol consumption,Exercise frequency
count,452.000000,452.000000,452.000000,452.000000,452.000000,452.000000,432.000000,427.000000,438.000000,446.000000
mean,40.285398,7.465708,0.788916,22.615044,52.823009,24.561947,1.641204,23.653396,1.173516,1.791480
std,13.172250,0.866625,0.135237,3.525963,15.654235,15.313665,1.356762,30.202785,1.621377,1.428134
min,9.000000,5.000000,0.500000,15.000000,18.000000,7.000000,0.000000,0.000000,0.000000,0.000000
25%,29.000000,7.000000,0.697500,20.000000,48.250000,15.000000,1.000000,0.000000,0.000000,0.000000
50%,40.000000,7.500000,0.820000,22.000000,58.000000,18.000000,1.000000,25.000000,0.000000,2.000000
75%,52.000000,8.000000,0.900000,25.000000,63.000000,32.500000,3.000000,50.000000,2.000000,3.000000
max,69.000000,10.000000,0.990000,30.000000,75.000000,63.000000,4.000000,200.000000,5.000000,5.000000


In [16]:
sleep_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 452 entries, 1 to 452
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Age                     452 non-null    int64  
 1   Gender                  452 non-null    object 
 2   Bedtime                 452 non-null    object 
 3   Wakeup time             452 non-null    object 
 4   Sleep duration          452 non-null    float64
 5   Sleep efficiency        452 non-null    float64
 6   REM sleep percentage    452 non-null    int64  
 7   Deep sleep percentage   452 non-null    int64  
 8   Light sleep percentage  452 non-null    int64  
 9   Awakenings              432 non-null    float64
 10  Caffeine consumption    427 non-null    float64
 11  Alcohol consumption     438 non-null    float64
 12  Smoking status          452 non-null    object 
 13  Exercise frequency      446 non-null    float64
dtypes: float64(6), int64(4), object(4)
memory usage

In [17]:
sleep_df["Sleep Quality"] = sleep_df.apply(
    lambda row: (
        min((row["Sleep duration"] * row["REM sleep percentage"] / 100) / 1.5, 1)
        + min((row["Sleep duration"] * row["Deep sleep percentage"] / 100) / 1.5, 1)
    )
    / 2 * 100,
    axis=1,
)
sleep_df

,Age,Gender,Bedtime,Wakeup time,Sleep duration,Sleep efficiency,REM sleep percentage,Deep sleep percentage,Light sleep percentage,Awakenings,Caffeine consumption,Alcohol consumption,Smoking status,Exercise frequency,Sleep Quality
ID,,,,,,,,,,,,,,,
1,65,Female,01:00:00,07:00:00,6.0,0.88,18,70,12,0.0,0.0,0.0,Yes,3.0,86.000000
2,69,Male,02:00:00,09:00:00,7.0,0.66,19,28,53,3.0,0.0,3.0,Yes,3.0,94.333333
3,40,Female,21:30:00,05:30:00,8.0,0.89,20,70,10,1.0,0.0,0.0,No,3.0,100.000000
4,40,Female,02:30:00,08:30:00,6.0,0.51,23,25,52,3.0,50.0,5.0,Yes,1.0,96.000000
5,57,Male,01:00:00,09:00:00,8.0,0.76,27,55,18,3.0,0.0,3.0,No,3.0,100.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
448,27,Female,22:00:00,05:30:00,7.5,0.91,22,57,21,0.0,0.0,0.0,No,5.0,100.000000
449,52,Male,21:00:00,03:00:00,6.0,0.74,28,57,15,4.0,25.0,0.0,No,3.0,100.000000
450,40,Female,23:00:00,07:30:00,8.5,0.55,20,32,48,1.0,NaN,3.0,Yes,0.0,100.000000


## Exploratory Data Analysis



In [18]:
import plotly.express as px

continous_columns = (
    "Bedtime",
    "Wakeup time",
    "Sleep duration",
    "Sleep efficiency",
    "REM sleep percentage",
    "Deep sleep percentage",
    "Light sleep percentage",
    "Awakenings",
    "Caffeine consumption",
    "Alcohol consumption",
    "Exercise frequency",
)

for col in continous_columns:
    fig1 = px.scatter(sleep_df, col, "Sleep Quality")
    fig1.show()

In [34]:
cat_columns = ("Age", "Gender", "Smoking status")

for col in cat_columns:
    fig2 = px.box(sleep_df, col, "Sleep Quality")
    fig2.show()